## Setup

Since this is a handin exercise I will shortly outline the setup required for this notebook to run (assuming I can only hand in the `.ipynb` file). The notebook pulls from the dependencies in the first code block. To install all relevant libraries run (assuming you have `jupyter` installed):

```shell
python -m pip install pandas tensorflow sklearn
```

Additionally it required the data and labels to be available as follows:
1. the batch data is located in a `data` folder next to the notebook
2. the label information is stored as `labels.csv` next to the notebook

In [ ]:
import pandas as pd
import numpy as np
import os
import math

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split


### Load Data

First the data needs to be loaded (first codeblock) and then go through some minor transformations before building the training, test and validation datasets. For this purpose all data is loaded into a flat dataframe that includes the batch and corresponding label in addition to the actual features `zeit`, `sensorid` und `messwert`. In preparation for the model training the `zeit` is transformed to float values and the data types of 

In [ ]:
# parse the labels.csv
labels = pd.read_csv('labels.csv', index_col=0)
labels = labels.sort_values('id')

# grab filenames from the data directory
filenames = os.listdir('data')
filenames.sort()

dataframes = []

# parse and concatenate all csv files into df
for filename in filenames:
  if filename.endswith('.csv'):
    batch = pd.read_csv(os.path.join('data',filename), index_col=0)
    batch['batch'] = int(filename.replace('.csv', ''))
    dataframes.append(batch)

df = pd.concat(dataframes, ignore_index=True)

# clean up original dataframes
del dataframes

# add label column (if it is not already available)
if (not 'label' in df.columns):
  df = df.merge(labels, left_on=["batch"], right_on=["id"])


In [ ]:
def time_to_float(inputstr):
  hours, minutes, seconds = map(float, inputstr.split(':'))

  # return hours * 3600 + minutes * 60 + seconds
  # this is sufficient because hours should always be 0
  return minutes * 60 + seconds

if (not df['sensorid'].dtype == 'int'):
  df['sensorid'] = df['sensorid'].astype('int')
if (not df['label'].dtype == 'category'):
  df['label'] = df['label'].astype('category')
if (not df['zeit'].dtype == 'float64'):
  df['zeit'] = df['zeit'].apply(time_to_float)

# print(df[:10])
# print(labels[:10])


### Prepare Training Data

In [ ]:
SEQUENCE_LENGTH = 256

sequences = []
sequence_labels = []

# build sequences based on chosen sequence length
for batch, readings in df.groupby('batch'):
  readings = readings.sort_values('zeit')
  for i in range(0, len(readings) - SEQUENCE_LENGTH, SEQUENCE_LENGTH):
    sequence = readings.iloc[i:i + SEQUENCE_LENGTH]
    sequences.append(sequence[['zeit', 'sensorid', 'messwert']].values)
    sequence_labels.append(sequence['label'].values[0])

# transform to numpy arrays for tensorflow
sequences = np.array(sequences)
sequence_labels = np.array(sequence_labels)

# split into train, validation and test sets
X_train, X_test, y_train, y_test = train_test_split(sequences, sequence_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


### Modelling & Training

In [ ]:
# make key hyperparameters accessible and easy to tune
BATCH_SIZE = 64
LSTM_UNITS = 256

N_BATCHES = math.ceil(len(X_train) / BATCH_SIZE)
num_features = X_train.shape[2]
num_classes = len(labels['label'].unique())

model = Sequential()
model.add(LSTM(LSTM_UNITS, input_shape=(SEQUENCE_LENGTH, num_features)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training
cp_cb = ModelCheckpoint(filepath='.checkpoints/cp-{epoch:03d}.ckpt', save_weights_only=True, save_freq=8*N_BATCHES)
stp_cb = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, min_delta=1e-4, start_from_epoch=32, verbose=1)

model.fit(X_train, y_train, epochs=256, batch_size=BATCH_SIZE, callbacks=[cp_cb, stp_cb], validation_data=(X_val, y_val))

# Save Model
model.save(f'models/lstm_{LSTM_UNITS}-seq_{SEQUENCE_LENGTH}-batch_{BATCH_SIZE}.keras')


### Evaluation

Best current model `lstm_256-seq_256-batch_64` (75.72%)

In [ ]:
print(model.predict(X_test[:5]))
loss, acc = model.evaluate(X_test, y_test, verbose=2)

print("Model accuracy: {:5.2f}%".format(100 * acc))
